## Develop a Deep Learning Based Churn Prediction Engine


### Import required libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score,recall_score, precision_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

#### Load and understand the data 

In [3]:
# Read the data
data = pd.read_csv("TelcoChurn.csv")

In [4]:
# Check the data types of the columns
data.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [5]:
# See the summary statistics
data.describe(include='all')

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
count,7043,7043,7043.000000,7043,7043,7043.000000,7043,7043,7043,7043,...,7043,7043,7043,7043,7043,7043,7043,7043.000000,7043,7043
unique,7043,2,NaN,2,2,NaN,2,2,3,2,...,2,2,2,2,3,2,4,NaN,6531,2
top,2460-FPSYH,Male,NaN,No,No,NaN,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,NaN,20.2,No
freq,1,3555,NaN,3641,4933,NaN,6361,4072,3096,5024,...,4621,4999,4336,4311,3875,4171,2365,NaN,11,5174
mean,NaN,NaN,0.162147,NaN,NaN,32.371149,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.761692,NaN,NaN
std,NaN,NaN,0.368612,NaN,NaN,24.559481,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.090047,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.250000,NaN,NaN
25%,NaN,NaN,0.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.500000,NaN,NaN
50%,NaN,NaN,0.000000,NaN,NaN,29.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.350000,NaN,NaN
75%,NaN,NaN,0.000000,NaN,NaN,55.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.850000,NaN,NaN


In [8]:
# Read the data
data = pd.read_csv("TelcoChurn.csv", na_values=' ')

In [9]:
# # Display the shape of the data
data.shape

(7043, 21)

In [10]:
# Display top 5 rows
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [11]:
# See the summary statistics
data.describe(include='all')

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
count,7043,7043,7043.000000,7043,7043,7043.000000,7043,7043,7043,7043,...,7043,7043,7043,7043,7043,7043,7043,7043.000000,7032.000000,7043
unique,7043,2,NaN,2,2,NaN,2,2,3,2,...,2,2,2,2,3,2,4,NaN,NaN,2
top,2460-FPSYH,Male,NaN,No,No,NaN,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,NaN,NaN,No
freq,1,3555,NaN,3641,4933,NaN,6361,4072,3096,5024,...,4621,4999,4336,4311,3875,4171,2365,NaN,NaN,5174
mean,NaN,NaN,0.162147,NaN,NaN,32.371149,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.761692,2283.300441,NaN
std,NaN,NaN,0.368612,NaN,NaN,24.559481,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.090047,2266.771362,NaN
min,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.250000,18.800000,NaN
25%,NaN,NaN,0.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.500000,401.450000,NaN
50%,NaN,NaN,0.000000,NaN,NaN,29.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.350000,1397.475000,NaN
75%,NaN,NaN,0.000000,NaN,NaN,55.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.850000,3794.737500,NaN


In [12]:
# Check the data types of the columns
data.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [13]:
# Check the number of unique levels in each column
data.nunique()

customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          2
InternetService        3
OnlineSecurity         2
OnlineBackup           2
DeviceProtection       2
TechSupport            2
StreamingTV            2
StreamingMovies        2
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6530
Churn                  2
dtype: int64

##### Check the distribution of target variable

In [14]:
# Let's see if there is a class imbalance in the target variable
print(data['Churn'].value_counts(normalize = True)*100)

No     73.463013
Yes    26.536987
Name: Churn, dtype: float64


In [15]:
# Check if there are any NA values in the data
data.isna().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

### Observations:

**On Type conversions:**

- Columns like customerID can be removed from the analysis
- The column 'SeniorCitizen' is a categorical column by its nature with 'Yes' as 1, and No as 0. So it should be converted into Categorical type

**On Missingness of data:**
 The data has few missing values in column TotalCharges
 
**On the class imbalance in the target attribute**
There are more instances where the customers didn't churn than those that have customers churned out. Class imbalance is slightly present.

## Data Pre-processing

In [16]:
# Remove customerID
data.drop(['customerID'], axis = 1, inplace=True)

### Type Conversions

In [17]:
# Convert 'SeniorCitizen' column into categorical
data['SeniorCitizen']= data['SeniorCitizen'].astype('category')

In [18]:
data.dtypes

gender                object
SeniorCitizen       category
Partner               object
Dependents            object
tenure                 int64
PhoneService          object
MultipleLines         object
InternetService       object
OnlineSecurity        object
OnlineBackup          object
DeviceProtection      object
TechSupport           object
StreamingTV           object
StreamingMovies       object
Contract              object
PaperlessBilling      object
PaymentMethod         object
MonthlyCharges       float64
TotalCharges         float64
Churn                 object
dtype: object

#### Preparing Data for Model building

In [19]:
data['Churn'] = data['Churn'].replace(to_replace=['No', 'Yes'], value=[0, 1])
data['Churn'].value_counts(normalize=True)

0    0.73463
1    0.26537
Name: Churn, dtype: float64

### Split the data into train and test sets


In [20]:
X = data.loc[:, data.columns != 'Churn']
y = data['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20, random_state=111, stratify = y)
print(X_train.shape, X_test.shape)

(5634, 19) (1409, 19)


In [21]:
# Check the Train and Test data distribution
print("Train data")
print(y_train.value_counts(normalize=True)*100)
print("\n")
print("Test data")
print(y_test.value_counts(normalize=True)*100)

Train data
0    73.464679
1    26.535321
Name: Churn, dtype: float64


Test data
0    73.456352
1    26.543648
Name: Churn, dtype: float64


### Split the attributes into numerical and categorical types

In [22]:
X_train.dtypes

gender                object
SeniorCitizen       category
Partner               object
Dependents            object
tenure                 int64
PhoneService          object
MultipleLines         object
InternetService       object
OnlineSecurity        object
OnlineBackup          object
DeviceProtection      object
TechSupport           object
StreamingTV           object
StreamingMovies       object
Contract              object
PaperlessBilling      object
PaymentMethod         object
MonthlyCharges       float64
TotalCharges         float64
dtype: object

In [23]:
num_attr=X_train.select_dtypes(['int64','float64']).columns
num_attr

Index(['tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')

In [24]:
cat_attr = X_train.select_dtypes(['category', 'object']).columns
cat_attr

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod'],
      dtype='object')

## Imputing missing values with median for numerical attributes

In [25]:
imputer = SimpleImputer(strategy='median')

imputer = imputer.fit(X_train[num_attr])
X_train[num_attr] = imputer.transform(X_train[num_attr])
X_test[num_attr] = imputer.transform(X_test[num_attr])

In [26]:
imputer.statistics_

array([  29.  ,   70.45, 1399.35])

### Imputing missing values with mode for categorical attributes

In [27]:
imputer = SimpleImputer(strategy='most_frequent')

imputer = imputer.fit(X_train[cat_attr])
X_train[cat_attr] = imputer.transform(X_train[cat_attr])
X_test[cat_attr] = imputer.transform(X_test[cat_attr])

In [28]:
imputer.statistics_

array(['Male', 0, 'No', 'No', 'Yes', 'No', 'Fiber optic', 'No', 'No',
       'No', 'No', 'No', 'No', 'Month-to-month', 'Yes',
       'Electronic check'], dtype=object)

### Standardizing the numerical attributes and One-hot encoding categorical attributes 

In [29]:
# DataFrameMapper, a class for mapping pandas data frame columns to different sklearn transformations
mapper = DataFrameMapper(
  [([continuous_col], StandardScaler()) for continuous_col in num_attr] +
  [([categorical_col], OneHotEncoder(handle_unknown='error')) for categorical_col in cat_attr]
, df_out=True)

In [30]:
mapper.fit(X_train)
X_train = mapper.transform(X_train)
X_test = mapper.transform(X_test)

In [31]:
X_train.head(10)

,tenure,MonthlyCharges,TotalCharges,gender_x0_Female,gender_x0_Male,SeniorCitizen_x0_0,SeniorCitizen_x0_1,Partner_x0_No,Partner_x0_Yes,Dependents_x0_No,...,StreamingMovies_x0_Yes,Contract_x0_Month-to-month,Contract_x0_One year,Contract_x0_Two year,PaperlessBilling_x0_No,PaperlessBilling_x0_Yes,PaymentMethod_x0_Bank transfer (automatic),PaymentMethod_x0_Credit card (automatic),PaymentMethod_x0_Electronic check,PaymentMethod_x0_Mailed check
3101,1.040564,0.499598,0.986096,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
6294,-1.280395,-1.495943,-0.998800,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
867,-1.117520,-1.469380,-0.961543,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4448,-0.099556,1.160351,0.299311,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4479,1.081283,-1.338225,-0.349932,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
4252,-0.547460,-0.287329,-0.552158,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2771,-0.791772,-0.053243,-0.647766,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1408,1.569906,1.306447,2.256793,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3559,-0.221712,-1.490962,-0.800493,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
5145,1.366313,-1.296721,-0.252408,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [32]:
print(X_train.shape, X_test.shape)

(5634, 39) (1409, 39)


In [33]:
print(y_train.shape, y_test.shape)

(5634,) (1409,)


### Defining Error Metrics

In [34]:
def get_CR_CM(train_actual,train_predicted,test_actual,test_predicted):
    print('''
             ========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
             ========================================
        ''')
    print(classification_report(train_actual, train_predicted, digits=4))

    print('''
             =========================================
               CLASSIFICATION REPORT FOR TEST DATA
             =========================================
            ''')
    print(classification_report(test_actual, test_predicted, digits=4))

    print('''
 ========================================
   Confusion Matrix FOR TRAIN DATA
 ========================================
            ''')
    print(confusion_matrix(train_actual, train_predicted))

    print('''
 =========================================
   Confusion matrix FOR TEST DATA
 =========================================
            ''')
    print(confusion_matrix(test_actual, test_predicted))
    

In [35]:
def get_ACCURACY_RECALL(train_actual,train_predicted,test_actual,test_predicted):
    print('''
 ========================================
           ACCURACY FOR TRAIN DATA
 ========================================
        ''')
    print("         ",accuracy_score(train_actual, train_predicted))

    print('''
 =========================================
           ACCURACY FOR TEST DATA
 =========================================
            ''')
    print("         ",accuracy_score(test_actual, test_predicted))

    print('''
 ========================================
           RECALL FOR TRAIN DATA
 ========================================
            ''')
    print("         ",recall_score(train_actual, train_predicted))

    print('''
 =========================================
           RECALL FOR TEST DATA
 =========================================
            ''')
    print("         ",recall_score(test_actual, test_predicted))
    

In [36]:
scores = pd.DataFrame(columns=['Model','Train_Accuracy','Train_Recall','Train_Precision','Train_F1_Score','Test_Accuracy','Test_Recall','Test_Precision','Test_F1_Score'])

def get_metrics(train_actual,train_predicted,test_actual,test_predicted,model_description,dataframe):
    train_accuracy = accuracy_score(train_actual,train_predicted)
    train_recall   = recall_score(train_actual,train_predicted)
    train_precision= precision_score(train_actual,train_predicted)
    train_f1score  = f1_score(train_actual,train_predicted)
    test_accuracy = accuracy_score(test_actual,test_predicted)
    test_recall   = recall_score(test_actual,test_predicted)
    test_precision= precision_score(test_actual,test_predicted)
    test_f1score  = f1_score(test_actual,test_predicted)
    dataframe = dataframe.append(pd.Series([model_description, train_accuracy,train_recall,train_precision,train_f1score,
                                            test_accuracy,test_recall,test_precision,test_f1score],
                                           index=scores.columns ), ignore_index=True)
    return(dataframe)

## Building the ANN Model

### Convert dataframe to numpy array to feed into Neural Networks

In [37]:
X_train = X_train.values
y_train = y_train.values
X_test = X_test.values
y_test = y_test.values

In [38]:
print(type(X_train))
print(type(X_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [39]:
X_train.shape

(5634, 39)

In [40]:
# Taking input_shape
input_shape = X_train.shape[1]
print(input_shape)

39


# Model 1 (1 hidden layer)

In [41]:
seed = 7
np.random.seed(seed)
tf.random.set_seed(seed)

In [42]:
model = Sequential()
model.add(Dense(32, input_dim=input_shape, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dense(1, activation='sigmoid',kernel_initializer='glorot_normal'))

In [43]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                1280      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,313
Trainable params: 1,313
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#### Callbacks - Early Stopping

In [45]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1, min_delta=0.001)

In [46]:
callbacks=[early_stopping]

In [47]:
model.fit(X_train, y_train,epochs=100,batch_size=64,validation_split=0.2,callbacks=callbacks)

Epoch 1/100
71/71 [==============================] - 0s 4ms/step - loss: 0.5230 - accuracy: 0.7428 - val_loss: 0.4357 - val_accuracy: 0.7986
Epoch 2/100
71/71 [==============================] - 0s 2ms/step - loss: 0.4401 - accuracy: 0.7914 - val_loss: 0.4154 - val_accuracy: 0.8048
Epoch 3/100
71/71 [==============================] - 0s 2ms/step - loss: 0.4297 - accuracy: 0.7939 - val_loss: 0.4138 - val_accuracy: 0.8119
Epoch 4/100
71/71 [==============================] - 0s 2ms/step - loss: 0.4262 - accuracy: 0.7981 - val_loss: 0.4054 - val_accuracy: 0.8119
Epoch 5/100
71/71 [==============================] - 0s 2ms/step - loss: 0.4231 - accuracy: 0.8016 - val_loss: 0.4068 - val_accuracy: 0.8137
Epoch 6/100
71/71 [==============================] - 0s 2ms/step - loss: 0.4218 - accuracy: 0.8034 - val_loss: 0.4033 - val_accuracy: 0.8146
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: 0.4202 - accuracy: 0.8021 - val_loss: 0.4083 - val_accuracy: 0.8154
Epoch 8/100
7

In [48]:
train_pred_1=model.predict_classes(X_train)
test_pred_1=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [49]:
get_CR_CM(y_train, train_pred_1, y_test, test_pred_1)


               CLASSIFICATION REPORT FOR TRAIN DATA
        
              precision    recall  f1-score   support

           0     0.8455    0.9019    0.8728      4139
           1     0.6669    0.5438    0.5991      1495

    accuracy                         0.8069      5634
   macro avg     0.7562    0.7229    0.7360      5634
weighted avg     0.7981    0.8069    0.8002      5634


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.8450    0.9005    0.8718      1035
           1     0.6634    0.5428    0.5971       374

    accuracy                         0.8055      1409
   macro avg     0.7542    0.7216    0.7345      1409
weighted avg     0.7968    0.8055    0.7989      1409


   Confusion Matrix FOR TRAIN DATA
            
[[3733  406]
 [ 682  813]]

   Confusion matrix FOR TEST DATA
            
[[932 103]
 [171 203]]


In [50]:
get_ACCURACY_RECALL(y_train, train_pred_1, y_test, test_pred_1)


           ACCURACY FOR TRAIN DATA
        
          0.8068867589634363

           ACCURACY FOR TEST DATA
            
          0.8055358410220014

           RECALL FOR TRAIN DATA
            
          0.5438127090301004

           RECALL FOR TEST DATA
            
          0.5427807486631016


In [51]:
scores = get_metrics(y_train,train_pred_1,y_test,test_pred_1,"1 layer model",scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,1 layer model,0.806887,0.543813,0.66694,0.599116,0.805536,0.542781,0.663399,0.597059


# Model 2 - Experiment 1 (2 hidden layers)

In [52]:
np.random.seed(seed)
tf.random.set_seed(seed)

In [53]:
model = Sequential()
model.add(Dense(16, input_dim=input_shape, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dense(12, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dense(1, activation='sigmoid',kernel_initializer='glorot_normal'))

In [54]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 16)                640       
_________________________________________________________________
dense_3 (Dense)              (None, 12)                204       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 13        
Total params: 857
Trainable params: 857
Non-trainable params: 0
_________________________________________________________________


In [55]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [56]:
model.fit(X_train, y_train, epochs=50, batch_size=64,validation_split=0.2,callbacks=callbacks)

Epoch 1/50
71/71 [==============================] - 0s 4ms/step - loss: 0.6686 - accuracy: 0.5682 - val_loss: 0.5173 - val_accuracy: 0.7533
Epoch 2/50
71/71 [==============================] - 0s 2ms/step - loss: 0.4932 - accuracy: 0.7555 - val_loss: 0.4403 - val_accuracy: 0.7924
Epoch 3/50
71/71 [==============================] - 0s 2ms/step - loss: 0.4460 - accuracy: 0.7910 - val_loss: 0.4263 - val_accuracy: 0.8110
Epoch 4/50
71/71 [==============================] - 0s 2ms/step - loss: 0.4358 - accuracy: 0.7928 - val_loss: 0.4133 - val_accuracy: 0.8004
Epoch 5/50
71/71 [==============================] - 0s 2ms/step - loss: 0.4304 - accuracy: 0.7950 - val_loss: 0.4131 - val_accuracy: 0.8083
Epoch 6/50
71/71 [==============================] - 0s 2ms/step - loss: 0.4274 - accuracy: 0.7963 - val_loss: 0.4095 - val_accuracy: 0.8110
Epoch 7/50
71/71 [==============================] - 0s 2ms/step - loss: 0.4247 - accuracy: 0.7976 - val_loss: 0.4197 - val_accuracy: 0.8119
Epoch 8/50
71/71 [==

In [57]:
train_pred_2=model.predict_classes(X_train)
test_pred_2=model.predict_classes(X_test)

In [58]:
get_CR_CM(y_train, train_pred_2, y_test, test_pred_2)


               CLASSIFICATION REPORT FOR TRAIN DATA
        
              precision    recall  f1-score   support

           0     0.8470    0.8947    0.8702      4139
           1     0.6545    0.5525    0.5992      1495

    accuracy                         0.8039      5634
   macro avg     0.7507    0.7236    0.7347      5634
weighted avg     0.7959    0.8039    0.7983      5634


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.8478    0.8986    0.8724      1035
           1     0.6635    0.5535    0.6035       374

    accuracy                         0.8070      1409
   macro avg     0.7556    0.7260    0.7380      1409
weighted avg     0.7988    0.8070    0.8010      1409


   Confusion Matrix FOR TRAIN DATA
            
[[3703  436]
 [ 669  826]]

   Confusion matrix FOR TEST DATA
            
[[930 105]
 [167 207]]


In [59]:
get_ACCURACY_RECALL(y_train, train_pred_2, y_test, test_pred_2)


           ACCURACY FOR TRAIN DATA
        
          0.80386936457224

           ACCURACY FOR TEST DATA
            
          0.8069552874378992

           RECALL FOR TRAIN DATA
            
          0.5525083612040134

           RECALL FOR TEST DATA
            
          0.553475935828877


In [60]:
scores = get_metrics(y_train,train_pred_2,y_test,test_pred_2,"2 layer model - experiment 1",scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,1 layer model,0.806887,0.543813,0.666940,0.599116,0.805536,0.542781,0.663399,0.597059
1,2 layer model - experiment 1,0.803869,0.552508,0.654517,0.599202,0.806955,0.553476,0.663462,0.603499


# Model 2 - Experiment 2 (2 hidden layers with different number of neurons)

In [61]:
np.random.seed(seed)
tf.random.set_seed(seed)

In [62]:
model = Sequential()
model.add(Dense(128, input_dim=input_shape, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dense(64, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dense(1, activation='sigmoid',kernel_initializer='glorot_normal'))

In [63]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 128)               5120      
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 13,441
Trainable params: 13,441
Non-trainable params: 0
_________________________________________________________________


In [64]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [65]:
model.fit(X_train, y_train, epochs=50, batch_size=64,validation_split=0.2,callbacks=callbacks)

Epoch 1/50
71/71 [==============================] - 0s 4ms/step - loss: 0.4696 - accuracy: 0.7695 - val_loss: 0.4186 - val_accuracy: 0.8004
Epoch 2/50
71/71 [==============================] - 0s 2ms/step - loss: 0.4244 - accuracy: 0.7976 - val_loss: 0.4248 - val_accuracy: 0.8030
Epoch 3/50
71/71 [==============================] - 0s 2ms/step - loss: 0.4202 - accuracy: 0.8010 - val_loss: 0.4124 - val_accuracy: 0.8190
Epoch 4/50
71/71 [==============================] - 0s 2ms/step - loss: 0.4153 - accuracy: 0.7974 - val_loss: 0.4099 - val_accuracy: 0.8163
Epoch 5/50
71/71 [==============================] - 0s 2ms/step - loss: 0.4133 - accuracy: 0.8045 - val_loss: 0.4098 - val_accuracy: 0.8172
Epoch 6/50
71/71 [==============================] - 0s 2ms/step - loss: 0.4088 - accuracy: 0.8054 - val_loss: 0.4082 - val_accuracy: 0.8154
Epoch 7/50
71/71 [==============================] - 0s 2ms/step - loss: 0.4061 - accuracy: 0.8103 - val_loss: 0.4046 - val_accuracy: 0.8039
Epoch 8/50
71/71 [==

In [66]:
train_pred_2=model.predict_classes(X_train)
test_pred_2=model.predict_classes(X_test)

In [67]:
get_CR_CM(y_train, train_pred_2, y_test, test_pred_2)


               CLASSIFICATION REPORT FOR TRAIN DATA
        
              precision    recall  f1-score   support

           0     0.8273    0.9423    0.8811      4139
           1     0.7402    0.4555    0.5640      1495

    accuracy                         0.8131      5634
   macro avg     0.7838    0.6989    0.7225      5634
weighted avg     0.8042    0.8131    0.7969      5634


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.8166    0.9333    0.8711      1035
           1     0.6947    0.4198    0.5233       374

    accuracy                         0.7970      1409
   macro avg     0.7556    0.6766    0.6972      1409
weighted avg     0.7842    0.7970    0.7788      1409


   Confusion Matrix FOR TRAIN DATA
            
[[3900  239]
 [ 814  681]]

   Confusion matrix FOR TEST DATA
            
[[966  69]
 [217 157]]


In [68]:
get_ACCURACY_RECALL(y_train, train_pred_2, y_test, test_pred_2)


           ACCURACY FOR TRAIN DATA
        
          0.8130990415335463

           ACCURACY FOR TEST DATA
            
          0.7970191625266146

           RECALL FOR TRAIN DATA
            
          0.45551839464882943

           RECALL FOR TEST DATA
            
          0.4197860962566845


In [69]:
scores = get_metrics(y_train,train_pred_2,y_test,test_pred_2,"2 layer model - experiment 2",scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,1 layer model,0.806887,0.543813,0.666940,0.599116,0.805536,0.542781,0.663399,0.597059
1,2 layer model - experiment 1,0.803869,0.552508,0.654517,0.599202,0.806955,0.553476,0.663462,0.603499
2,2 layer model - experiment 2,0.813099,0.455518,0.740217,0.563975,0.797019,0.419786,0.694690,0.523333


# Model 3 (2 hidden layers with Dropout)

In [70]:
np.random.seed(seed)
tf.random.set_seed(seed)

In [71]:
model = Sequential()
model.add(Dense(128, input_dim=input_shape, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid',kernel_initializer='glorot_normal'))

In [72]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 128)               5120      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total params: 13,441
Trainable params: 13,441
Non-trainable params: 0
_________________________________________________________________


In [73]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [74]:
model.fit(X_train, y_train,
              epochs=50,
          batch_size=64,validation_split=0.2,callbacks=callbacks)

Epoch 1/50
71/71 [==============================] - 0s 4ms/step - loss: 0.5320 - accuracy: 0.7295 - val_loss: 0.4238 - val_accuracy: 0.8075
Epoch 2/50
71/71 [==============================] - 0s 3ms/step - loss: 0.4660 - accuracy: 0.7752 - val_loss: 0.4158 - val_accuracy: 0.8066
Epoch 3/50
71/71 [==============================] - 0s 3ms/step - loss: 0.4520 - accuracy: 0.7841 - val_loss: 0.4184 - val_accuracy: 0.8057
Epoch 4/50
71/71 [==============================] - 0s 3ms/step - loss: 0.4390 - accuracy: 0.7870 - val_loss: 0.4075 - val_accuracy: 0.8119
Epoch 5/50
71/71 [==============================] - 0s 3ms/step - loss: 0.4436 - accuracy: 0.7859 - val_loss: 0.4115 - val_accuracy: 0.8110
Epoch 6/50
71/71 [==============================] - 0s 3ms/step - loss: 0.4438 - accuracy: 0.7897 - val_loss: 0.4079 - val_accuracy: 0.8110
Epoch 7/50
71/71 [==============================] - 0s 3ms/step - loss: 0.4375 - accuracy: 0.7934 - val_loss: 0.4197 - val_accuracy: 0.8128
Epoch 8/50
71/71 [==

In [75]:
train_pred_3=model.predict_classes(X_train)
test_pred_3=model.predict_classes(X_test)

In [76]:
get_CR_CM(y_train, train_pred_3, y_test, test_pred_3)


               CLASSIFICATION REPORT FOR TRAIN DATA
        
              precision    recall  f1-score   support

           0     0.8480    0.9005    0.8734      4139
           1     0.6675    0.5532    0.6050      1495

    accuracy                         0.8083      5634
   macro avg     0.7577    0.7268    0.7392      5634
weighted avg     0.8001    0.8083    0.8022      5634


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.8398    0.8966    0.8673      1035
           1     0.6480    0.5267    0.5811       374

    accuracy                         0.7984      1409
   macro avg     0.7439    0.7117    0.7242      1409
weighted avg     0.7889    0.7984    0.7913      1409


   Confusion Matrix FOR TRAIN DATA
            
[[3727  412]
 [ 668  827]]

   Confusion matrix FOR TEST DATA
            
[[928 107]
 [177 197]]


In [77]:
get_ACCURACY_RECALL(y_train, train_pred_3, y_test, test_pred_3)


           ACCURACY FOR TRAIN DATA
        
          0.8083067092651757

           ACCURACY FOR TEST DATA
            
          0.7984386089425124

           RECALL FOR TRAIN DATA
            
          0.5531772575250836

           RECALL FOR TEST DATA
            
          0.5267379679144385


In [78]:
scores = get_metrics(y_train,train_pred_3,y_test,test_pred_3,"2 layer model with dropout",scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,1 layer model,0.806887,0.543813,0.666940,0.599116,0.805536,0.542781,0.663399,0.597059
1,2 layer model - experiment 1,0.803869,0.552508,0.654517,0.599202,0.806955,0.553476,0.663462,0.603499
2,2 layer model - experiment 2,0.813099,0.455518,0.740217,0.563975,0.797019,0.419786,0.694690,0.523333
3,2 layer model with dropout,0.808307,0.553177,0.667474,0.604974,0.798439,0.526738,0.648026,0.581121


# Model 4 (2 hidden layers with l2 regularizer)

In [79]:
np.random.seed(seed)
tf.random.set_seed(seed)

In [80]:
model = Sequential()
model.add(Dense(64, input_dim=input_shape, kernel_initializer='glorot_normal', activation='relu',kernel_regularizer=l2(1e-03)))
model.add(Dropout(0.5))
model.add(Dense(32, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid',kernel_initializer='glorot_normal'))

In [81]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 64)                2560      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 33        
Total params: 4,673
Trainable params: 4,673
Non-trainable params: 0
_________________________________________________________________


In [82]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [83]:
model.fit(X_train, y_train,
              epochs=50,
          batch_size=64,validation_split=0.2,callbacks=callbacks)

Epoch 1/50
71/71 [==============================] - 0s 4ms/step - loss: 0.6672 - accuracy: 0.6741 - val_loss: 0.5189 - val_accuracy: 0.7720
Epoch 2/50
71/71 [==============================] - 0s 3ms/step - loss: 0.5485 - accuracy: 0.7559 - val_loss: 0.4663 - val_accuracy: 0.8030
Epoch 3/50
71/71 [==============================] - 0s 3ms/step - loss: 0.5146 - accuracy: 0.7799 - val_loss: 0.4600 - val_accuracy: 0.8075
Epoch 4/50
71/71 [==============================] - 0s 3ms/step - loss: 0.4997 - accuracy: 0.7735 - val_loss: 0.4517 - val_accuracy: 0.8075
Epoch 5/50
71/71 [==============================] - 0s 3ms/step - loss: 0.4963 - accuracy: 0.7786 - val_loss: 0.4485 - val_accuracy: 0.8110
Epoch 6/50
71/71 [==============================] - ETA: 0s - loss: 0.4970 - accuracy: 0.77 - 0s 3ms/step - loss: 0.4910 - accuracy: 0.7828 - val_loss: 0.4442 - val_accuracy: 0.8154
Epoch 7/50
71/71 [==============================] - 0s 3ms/step - loss: 0.4852 - accuracy: 0.7830 - val_loss: 0.4505 -

In [84]:
train_pred_5=model.predict_classes(X_train)
test_pred_5=model.predict_classes(X_test)

In [85]:
get_CR_CM(y_train, train_pred_5, y_test, test_pred_5)


               CLASSIFICATION REPORT FOR TRAIN DATA
        
              precision    recall  f1-score   support

           0     0.8503    0.9029    0.8758      4139
           1     0.6755    0.5599    0.6123      1495

    accuracy                         0.8119      5634
   macro avg     0.7629    0.7314    0.7440      5634
weighted avg     0.8039    0.8119    0.8059      5634


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.8409    0.8986    0.8688      1035
           1     0.6535    0.5294    0.5849       374

    accuracy                         0.8006      1409
   macro avg     0.7472    0.7140    0.7268      1409
weighted avg     0.7911    0.8006    0.7934      1409


   Confusion Matrix FOR TRAIN DATA
            
[[3737  402]
 [ 658  837]]

   Confusion matrix FOR TEST DATA
            
[[930 105]
 [176 198]]


In [86]:
get_ACCURACY_RECALL(y_train, train_pred_5, y_test, test_pred_5)


           ACCURACY FOR TRAIN DATA
        
          0.8118565850195243

           ACCURACY FOR TEST DATA
            
          0.8005677785663591

           RECALL FOR TRAIN DATA
            
          0.559866220735786

           RECALL FOR TEST DATA
            
          0.5294117647058824


In [87]:
scores = get_metrics(y_train,train_pred_5,y_test,test_pred_5,"2 layer model with dropout with L2 regularizer",scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,1 layer model,0.806887,0.543813,0.666940,0.599116,0.805536,0.542781,0.663399,0.597059
1,2 layer model - experiment 1,0.803869,0.552508,0.654517,0.599202,0.806955,0.553476,0.663462,0.603499
2,2 layer model - experiment 2,0.813099,0.455518,0.740217,0.563975,0.797019,0.419786,0.694690,0.523333
3,2 layer model with dropout,0.808307,0.553177,0.667474,0.604974,0.798439,0.526738,0.648026,0.581121
4,2 layer model with dropout with L2 regularizer,0.811857,0.559866,0.675545,0.612290,0.800568,0.529412,0.653465,0.584934


# Model 5 (2 hidden layers with l2 regularizer and class_weight)

In [88]:
np.random.seed(seed)
tf.random.set_seed(seed)

In [89]:
model = Sequential()
model.add(Dense(128, input_dim=input_shape, kernel_initializer='glorot_normal', activation='relu',kernel_regularizer=l2(1e-03)))
model.add(Dropout(0.5))
model.add(Dense(64, kernel_initializer='glorot_normal', activation='relu',kernel_regularizer=l2(1e-03)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid',kernel_initializer='glorot_normal'))

In [90]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 128)               5120      
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 65        
Total params: 13,441
Trainable params: 13,441
Non-trainable params: 0
_________________________________________________________________


In [91]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [92]:
model.fit(X_train, y_train,
              epochs=50,
          batch_size=64, class_weight={0:0.25, 1:0.75},validation_split=0.2, callbacks=callbacks)

Epoch 1/50
71/71 [==============================] - 0s 4ms/step - loss: 0.3606 - accuracy: 0.6470 - val_loss: 0.6371 - val_accuracy: 0.7498
Epoch 2/50
71/71 [==============================] - 0s 2ms/step - loss: 0.3057 - accuracy: 0.7187 - val_loss: 0.6087 - val_accuracy: 0.7445
Epoch 3/50
71/71 [==============================] - 0s 2ms/step - loss: 0.2811 - accuracy: 0.7207 - val_loss: 0.6171 - val_accuracy: 0.7161
Epoch 4/50
71/71 [==============================] - 0s 2ms/step - loss: 0.2604 - accuracy: 0.7260 - val_loss: 0.5508 - val_accuracy: 0.7622
Epoch 5/50
71/71 [==============================] - 0s 2ms/step - loss: 0.2530 - accuracy: 0.7302 - val_loss: 0.5761 - val_accuracy: 0.7232
Epoch 6/50
71/71 [==============================] - 0s 2ms/step - loss: 0.2452 - accuracy: 0.7251 - val_loss: 0.5505 - val_accuracy: 0.7427
Epoch 7/50
71/71 [==============================] - 0s 2ms/step - loss: 0.2373 - accuracy: 0.7244 - val_loss: 0.6019 - val_accuracy: 0.6877
Epoch 8/50
71/71 [==

In [93]:
train_pred_4=model.predict_classes(X_train)
test_pred_4=model.predict_classes(X_test)

In [94]:
get_CR_CM(y_train, train_pred_4, y_test, test_pred_4)


               CLASSIFICATION REPORT FOR TRAIN DATA
        
              precision    recall  f1-score   support

           0     0.9236    0.6922    0.7913      4139
           1     0.4968    0.8415    0.6248      1495

    accuracy                         0.7318      5634
   macro avg     0.7102    0.7668    0.7081      5634
weighted avg     0.8104    0.7318    0.7471      5634


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.9206    0.6947    0.7919      1035
           1     0.4968    0.8342    0.6228       374

    accuracy                         0.7317      1409
   macro avg     0.7087    0.7645    0.7073      1409
weighted avg     0.8081    0.7317    0.7470      1409


   Confusion Matrix FOR TRAIN DATA
            
[[2865 1274]
 [ 237 1258]]

   Confusion matrix FOR TEST DATA
            
[[719 316]
 [ 62 312]]


In [95]:
get_ACCURACY_RECALL(y_train, train_pred_4, y_test, test_pred_4)


           ACCURACY FOR TRAIN DATA
        
          0.7318068867589634

           ACCURACY FOR TEST DATA
            
          0.7317246273953159

           RECALL FOR TRAIN DATA
            
          0.8414715719063545

           RECALL FOR TEST DATA
            
          0.8342245989304813


In [96]:
scores = get_metrics(y_train,train_pred_4,y_test,test_pred_4,"2 layer model with dropout and class weight",scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,1 layer model,0.806887,0.543813,0.666940,0.599116,0.805536,0.542781,0.663399,0.597059
1,2 layer model - experiment 1,0.803869,0.552508,0.654517,0.599202,0.806955,0.553476,0.663462,0.603499
2,2 layer model - experiment 2,0.813099,0.455518,0.740217,0.563975,0.797019,0.419786,0.694690,0.523333
3,2 layer model with dropout,0.808307,0.553177,0.667474,0.604974,0.798439,0.526738,0.648026,0.581121
4,2 layer model with dropout with L2 regularizer,0.811857,0.559866,0.675545,0.612290,0.800568,0.529412,0.653465,0.584934
5,2 layer model with dropout and class weight,0.731807,0.841472,0.496840,0.624783,0.731725,0.834225,0.496815,0.622754


# Model 6 (2 hidden layers with l1 regularizer)

In [97]:
np.random.seed(seed)
tf.random.set_seed(seed)

In [98]:
model = Sequential()
model.add(Dense(128, input_dim=input_shape, kernel_initializer='glorot_normal', activation='relu',kernel_regularizer=l1(0.01)))
model.add(Dropout(0.5))
model.add(Dense(64, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid',kernel_initializer='glorot_normal'))


In [99]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [100]:
warnings.filterwarnings("ignore")

In [101]:
model.fit(X_train, y_train,
              epochs=50,
          batch_size=64, class_weight={0:0.25, 1:0.75},validation_split=0.2,callbacks=callbacks)

Epoch 1/50
71/71 [==============================] - 0s 3ms/step - loss: 3.3025 - accuracy: 0.6037 - val_loss: 2.4355 - val_accuracy: 0.7063
Epoch 2/50
71/71 [==============================] - 0s 2ms/step - loss: 1.3469 - accuracy: 0.6927 - val_loss: 1.1346 - val_accuracy: 0.7028
Epoch 3/50
71/71 [==============================] - 0s 2ms/step - loss: 0.5060 - accuracy: 0.7118 - val_loss: 0.7159 - val_accuracy: 0.6531
Epoch 4/50
71/71 [==============================] - 0s 2ms/step - loss: 0.2753 - accuracy: 0.7002 - val_loss: 0.5475 - val_accuracy: 0.7773
Epoch 5/50
71/71 [==============================] - 0s 2ms/step - loss: 0.2539 - accuracy: 0.6978 - val_loss: 0.5596 - val_accuracy: 0.7303
Epoch 6/50
71/71 [==============================] - 0s 2ms/step - loss: 0.2503 - accuracy: 0.6958 - val_loss: 0.6321 - val_accuracy: 0.6637
Epoch 7/50
71/71 [==============================] - 0s 2ms/step - loss: 0.2504 - accuracy: 0.7014 - val_loss: 0.5266 - val_accuracy: 0.7489
Epoch 8/50
71/71 [==

In [102]:
train_pred_6=model.predict_classes(X_train)
test_pred_6=model.predict_classes(X_test)

In [103]:
get_CR_CM(y_train, train_pred_6, y_test, test_pred_6)


               CLASSIFICATION REPORT FOR TRAIN DATA
        
              precision    recall  f1-score   support

           0     0.9090    0.7236    0.8058      4139
           1     0.5109    0.7993    0.6234      1495

    accuracy                         0.7437      5634
   macro avg     0.7099    0.7615    0.7146      5634
weighted avg     0.8033    0.7437    0.7574      5634


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.9069    0.7246    0.8056      1035
           1     0.5103    0.7941    0.6213       374

    accuracy                         0.7431      1409
   macro avg     0.7086    0.7594    0.7135      1409
weighted avg     0.8016    0.7431    0.7567      1409


   Confusion Matrix FOR TRAIN DATA
            
[[2995 1144]
 [ 300 1195]]

   Confusion matrix FOR TEST DATA
            
[[750 285]
 [ 77 297]]


In [104]:
get_ACCURACY_RECALL(y_train, train_pred_6, y_test, test_pred_6)


           ACCURACY FOR TRAIN DATA
        
          0.7436989705360313

           ACCURACY FOR TEST DATA
            
          0.7430801987224982

           RECALL FOR TRAIN DATA
            
          0.7993311036789298

           RECALL FOR TEST DATA
            
          0.7941176470588235


In [105]:
scores = get_metrics(y_train,train_pred_6,y_test,test_pred_6,"2 layer model with dropout with L1 regularizer",scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,1 layer model,0.806887,0.543813,0.666940,0.599116,0.805536,0.542781,0.663399,0.597059
1,2 layer model - experiment 1,0.803869,0.552508,0.654517,0.599202,0.806955,0.553476,0.663462,0.603499
2,2 layer model - experiment 2,0.813099,0.455518,0.740217,0.563975,0.797019,0.419786,0.694690,0.523333
3,2 layer model with dropout,0.808307,0.553177,0.667474,0.604974,0.798439,0.526738,0.648026,0.581121
4,2 layer model with dropout with L2 regularizer,0.811857,0.559866,0.675545,0.612290,0.800568,0.529412,0.653465,0.584934
5,2 layer model with dropout and class weight,0.731807,0.841472,0.496840,0.624783,0.731725,0.834225,0.496815,0.622754
6,2 layer model with dropout with L1 regularizer,0.743699,0.799331,0.510902,0.623370,0.743080,0.794118,0.510309,0.621339


# Model 7

## MLP using features from AutoEncoders

In [106]:
encoding_dim  = 16 #8,12, 16, 20
# this is our input placeholder
input_data = Input(shape=(input_shape,))

# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_data)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(input_shape, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(inputs=input_data, outputs=decoded)

In [107]:
autoencoder.compile(optimizer='adam', loss='mse')

In [108]:
autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=64,
                shuffle=True,
                validation_data=(X_test, X_test), callbacks=callbacks)

Epoch 1/50
89/89 [==============================] - 0s 2ms/step - loss: 0.2984 - val_loss: 0.2483
Epoch 2/50
89/89 [==============================] - 0s 1ms/step - loss: 0.2160 - val_loss: 0.1905
Epoch 3/50
89/89 [==============================] - 0s 1ms/step - loss: 0.1726 - val_loss: 0.1570
Epoch 4/50
89/89 [==============================] - 0s 1ms/step - loss: 0.1444 - val_loss: 0.1336
Epoch 5/50
89/89 [==============================] - 0s 1ms/step - loss: 0.1238 - val_loss: 0.1155
Epoch 6/50
89/89 [==============================] - 0s 1ms/step - loss: 0.1077 - val_loss: 0.1014
Epoch 7/50
89/89 [==============================] - 0s 1ms/step - loss: 0.0950 - val_loss: 0.0901
Epoch 8/50
89/89 [==============================] - 0s 1ms/step - loss: 0.0849 - val_loss: 0.0808
Epoch 9/50
89/89 [==============================] - 0s 1ms/step - loss: 0.0766 - val_loss: 0.0734
Epoch 10/50
89/89 [==============================] - 0s 1ms/step - loss: 0.0701 - val_loss: 0.0677
Epoch 11/50
89/89 [

In [109]:
# this model maps an input to its encoded representation
encoder = Model(inputs=input_data, outputs=encoded)

In [110]:
x_train_encoded = encoder.predict(X_train)
x_test_encoded = encoder.predict(X_test)

In [111]:
x_train_encoded.shape

(5634, 16)

In [112]:
model = Sequential()

model.add(Dense(64, input_dim = encoding_dim, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))

In [113]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 64)                1088      
_________________________________________________________________
dense_23 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 33        
Total params: 3,201
Trainable params: 3,201
Non-trainable params: 0
_________________________________________________________________


In [114]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [115]:
model.fit(x_train_encoded, y_train, batch_size=64, epochs=50,validation_split=0.2,callbacks=callbacks)

Epoch 1/50
71/71 [==============================] - ETA: 0s - loss: 0.5384 - accuracy: 0.72 - 0s 3ms/step - loss: 0.5134 - accuracy: 0.7435 - val_loss: 0.4270 - val_accuracy: 0.7933
Epoch 2/50
71/71 [==============================] - 0s 1ms/step - loss: 0.4397 - accuracy: 0.7859 - val_loss: 0.4430 - val_accuracy: 0.7950
Epoch 3/50
71/71 [==============================] - 0s 1ms/step - loss: 0.4363 - accuracy: 0.7819 - val_loss: 0.4400 - val_accuracy: 0.7906
Epoch 4/50
71/71 [==============================] - 0s 1ms/step - loss: 0.4282 - accuracy: 0.7939 - val_loss: 0.4095 - val_accuracy: 0.8119
Epoch 5/50
71/71 [==============================] - 0s 1ms/step - loss: 0.4262 - accuracy: 0.7996 - val_loss: 0.4147 - val_accuracy: 0.8128
Epoch 6/50
71/71 [==============================] - 0s 1ms/step - loss: 0.4245 - accuracy: 0.7959 - val_loss: 0.4184 - val_accuracy: 0.8083
Epoch 7/50
71/71 [==============================] - 0s 1ms/step - loss: 0.4230 - accuracy: 0.7974 - val_loss: 0.4146 -

In [116]:
train_pred_7=model.predict_classes(x_train_encoded)
test_pred_7=model.predict_classes(x_test_encoded)

In [117]:
get_CR_CM(y_train, train_pred_7, y_test, test_pred_7)


               CLASSIFICATION REPORT FOR TRAIN DATA
        
              precision    recall  f1-score   support

           0     0.8627    0.8649    0.8638      4139
           1     0.6233    0.6187    0.6210      1495

    accuracy                         0.7996      5634
   macro avg     0.7430    0.7418    0.7424      5634
weighted avg     0.7991    0.7996    0.7994      5634


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.8574    0.8541    0.8558      1035
           1     0.6005    0.6070    0.6037       374

    accuracy                         0.7885      1409
   macro avg     0.7290    0.7305    0.7297      1409
weighted avg     0.7892    0.7885    0.7889      1409


   Confusion Matrix FOR TRAIN DATA
            
[[3580  559]
 [ 570  925]]

   Confusion matrix FOR TEST DATA
            
[[884 151]
 [147 227]]


In [118]:
get_ACCURACY_RECALL(y_train, train_pred_7, y_test, test_pred_7)


           ACCURACY FOR TRAIN DATA
        
          0.7996095136670217

           ACCURACY FOR TEST DATA
            
          0.7885024840312278

           RECALL FOR TRAIN DATA
            
          0.6187290969899666

           RECALL FOR TEST DATA
            
          0.606951871657754


In [119]:
scores = get_metrics(y_train,train_pred_7,y_test,test_pred_7,"autoencoder with 2 layer model",scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,1 layer model,0.806887,0.543813,0.666940,0.599116,0.805536,0.542781,0.663399,0.597059
1,2 layer model - experiment 1,0.803869,0.552508,0.654517,0.599202,0.806955,0.553476,0.663462,0.603499
2,2 layer model - experiment 2,0.813099,0.455518,0.740217,0.563975,0.797019,0.419786,0.694690,0.523333
3,2 layer model with dropout,0.808307,0.553177,0.667474,0.604974,0.798439,0.526738,0.648026,0.581121
4,2 layer model with dropout with L2 regularizer,0.811857,0.559866,0.675545,0.612290,0.800568,0.529412,0.653465,0.584934
5,2 layer model with dropout and class weight,0.731807,0.841472,0.496840,0.624783,0.731725,0.834225,0.496815,0.622754
6,2 layer model with dropout with L1 regularizer,0.743699,0.799331,0.510902,0.623370,0.743080,0.794118,0.510309,0.621339
7,autoencoder with 2 layer model,0.799610,0.618729,0.623315,0.621014,0.788502,0.606952,0.600529,0.603723


# Autoencoders with class weight

In [120]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 64)                1088      
_________________________________________________________________
dense_23 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 33        
Total params: 3,201
Trainable params: 3,201
Non-trainable params: 0
_________________________________________________________________


In [121]:
model.fit(x_train_encoded, y_train, batch_size=64, epochs=50, class_weight={0:0.25, 1:0.75},validation_split=0.2,callbacks=callbacks)

Epoch 1/50
71/71 [==============================] - 0s 2ms/step - loss: 0.1858 - accuracy: 0.7280 - val_loss: 0.5586 - val_accuracy: 0.6868
Epoch 2/50
71/71 [==============================] - 0s 2ms/step - loss: 0.1830 - accuracy: 0.7411 - val_loss: 0.5723 - val_accuracy: 0.6735
Epoch 3/50
71/71 [==============================] - 0s 2ms/step - loss: 0.1838 - accuracy: 0.7373 - val_loss: 0.5387 - val_accuracy: 0.7098
Epoch 4/50
71/71 [==============================] - 0s 2ms/step - loss: 0.1825 - accuracy: 0.7404 - val_loss: 0.5484 - val_accuracy: 0.7028
Epoch 5/50
71/71 [==============================] - 0s 2ms/step - loss: 0.1818 - accuracy: 0.7384 - val_loss: 0.5273 - val_accuracy: 0.7240
Epoch 6/50
71/71 [==============================] - 0s 2ms/step - loss: 0.1815 - accuracy: 0.7417 - val_loss: 0.5599 - val_accuracy: 0.6877
Epoch 7/50
71/71 [==============================] - 0s 2ms/step - loss: 0.1811 - accuracy: 0.7364 - val_loss: 0.4809 - val_accuracy: 0.7631
Epoch 8/50
71/71 [==

In [122]:
train_pred_8=model.predict_classes(x_train_encoded)
test_pred_8=model.predict_classes(x_test_encoded)

In [123]:
get_CR_CM(y_train, train_pred_8, y_test, test_pred_8)


               CLASSIFICATION REPORT FOR TRAIN DATA
        
              precision    recall  f1-score   support

           0     0.9074    0.7652    0.8303      4139
           1     0.5466    0.7839    0.6441      1495

    accuracy                         0.7701      5634
   macro avg     0.7270    0.7746    0.7372      5634
weighted avg     0.8117    0.7701    0.7809      5634


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.9019    0.7643    0.8274      1035
           1     0.5414    0.7701    0.6358       374

    accuracy                         0.7658      1409
   macro avg     0.7216    0.7672    0.7316      1409
weighted avg     0.8062    0.7658    0.7765      1409


   Confusion Matrix FOR TRAIN DATA
            
[[3167  972]
 [ 323 1172]]

   Confusion matrix FOR TEST DATA
            
[[791 244]
 [ 86 288]]


In [124]:
get_ACCURACY_RECALL(y_train, train_pred_8, y_test, test_pred_8)


           ACCURACY FOR TRAIN DATA
        
          0.7701455449059283

           ACCURACY FOR TEST DATA
            
          0.765791341376863

           RECALL FOR TRAIN DATA
            
          0.7839464882943143

           RECALL FOR TEST DATA
            
          0.7700534759358288


In [125]:
scores = get_metrics(y_train,train_pred_8,y_test,test_pred_8,"autoencoder with 2 layer model and with class weight",scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,1 layer model,0.806887,0.543813,0.666940,0.599116,0.805536,0.542781,0.663399,0.597059
1,2 layer model - experiment 1,0.803869,0.552508,0.654517,0.599202,0.806955,0.553476,0.663462,0.603499
2,2 layer model - experiment 2,0.813099,0.455518,0.740217,0.563975,0.797019,0.419786,0.694690,0.523333
3,2 layer model with dropout,0.808307,0.553177,0.667474,0.604974,0.798439,0.526738,0.648026,0.581121
4,2 layer model with dropout with L2 regularizer,0.811857,0.559866,0.675545,0.612290,0.800568,0.529412,0.653465,0.584934
5,2 layer model with dropout and class weight,0.731807,0.841472,0.496840,0.624783,0.731725,0.834225,0.496815,0.622754
6,2 layer model with dropout with L1 regularizer,0.743699,0.799331,0.510902,0.623370,0.743080,0.794118,0.510309,0.621339
7,autoencoder with 2 layer model,0.799610,0.618729,0.623315,0.621014,0.788502,0.606952,0.600529,0.603723
8,autoencoder with 2 layer model and with class ...,0.770146,0.783946,0.546642,0.644133,0.765791,0.770053,0.541353,0.635762
